In [64]:
import pandas as pd

In [65]:
# Standard Satellite Catalog: Objects from the US Satellite Catalog
satcat_df = pd.read_csv('../data/satcat.tsv', sep='\t').drop(0, axis=0)

C:\Users\dfarr\AppData\Local\Temp\ipykernel_14656\3262813254.py:2: DtypeWarning: Columns (1,18,20,22,24,26,28,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  satcat_df = pd.read_csv('../data/satcat.tsv', sep='\t').drop(0, axis=0)


In [66]:
# Auxiliary Satellite Catalog: Objects omitted from the US catalog
auxcat_df = pd.read_csv('../data/auxcat.tsv', sep='\t').drop(0, axis=0)

In [67]:
# Failed Satellite Catalog: Objects which failed to reach orbit
ftocat_df = pd.read_csv('../data/ftocat.tsv', sep='\t').drop(0, axis=0)

In [68]:
# Organizations Catalog:  The country of origin, owner organization, and/or manufacturer of the object of interest
orgs_df = pd.read_csv('../data/orgs.tsv', sep='\t').drop(0, axis=0)

In [69]:
satcat_df.shape

(60057, 41)

In [70]:
auxcat_df.shape

(11235, 41)

In [71]:
ftocat_df.shape

(1695, 41)

In [72]:
orgs_df.shape

(3707, 17)

In [73]:
# Object Catalogs Total Rows
objcat_rows_total = satcat_df.shape[0] + auxcat_df.shape[0] + ftocat_df.shape[0]
objcat_shape = (objcat_rows_total, 41)
objcat_shape

(72987, 41)

In [74]:
# Confirming that auxcat and satcat have the same columns before joining
satcat_df.columns == auxcat_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [75]:
# All active artificial objects in orbit
all_obj_df = pd.concat([satcat_df, auxcat_df], axis=0)

In [76]:
all_obj_df.columns

Index(['#JCAT', 'Satcat', 'Piece', 'Type', 'Name', 'PLName', 'LDate', 'Parent',
       'SDate', 'Primary', 'DDate', 'Status', 'Dest', 'Owner', 'State',
       'Manufacturer', 'Bus', 'Motor', 'Mass', 'MassFlag', 'DryMass',
       'DryFlag', 'TotMass', 'TotFlag', 'Length', 'LFlag', 'Diameter', 'DFlag',
       'Span', 'SpanFlag', 'Shape', 'ODate', 'Perigee', 'PF', 'Apogee', 'AF',
       'Inc', 'IF', 'OpOrbit', 'OQUAL', 'AltNames'],
      dtype='object')

In [77]:
# Convert all date columns to datetime
all_obj_df['LDate'] = pd.to_datetime(all_obj_df['LDate'], errors='coerce')
all_obj_df['SDate'] = pd.to_datetime(all_obj_df['SDate'], errors='coerce')
all_obj_df['DDate'] = pd.to_datetime(all_obj_df['DDate'], errors='coerce')
all_obj_df['ODate'] = pd.to_datetime(all_obj_df['ODate'], errors='coerce')

C:\Users\dfarr\AppData\Local\Temp\ipykernel_14656\2993661639.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_obj_df['DDate'] = pd.to_datetime(all_obj_df['DDate'], errors='coerce')


In [78]:
all_obj_df[['LDate', 'SDate', 'DDate', 'ODate']].describe()

,LDate,SDate,DDate,ODate
count,71139,37914,33531,71013
mean,1995-01-10 21:22:45.580061568,1996-07-28 03:05:20.841905280,1996-08-01 07:09:39.216247552,2000-03-09 11:52:40.170672896
min,1957-10-04 00:00:00,1957-10-04 19:33:00,1957-10-17 00:00:00,1957-10-04 00:00:00
25%,1979-08-10 00:00:00,1981-07-24 23:51:00,1981-03-07 12:00:00,1985-07-12 00:00:00
50%,1993-06-16 00:00:00,2000-03-11 13:04:00,1994-12-31 00:00:00,2003-10-17 00:00:00
75%,2012-12-19 00:00:00,2009-02-10 16:56:00,2013-12-04 00:00:00,2018-10-01 00:00:00
max,2024-06-08 00:00:00,2024-06-05 04:08:00,2024-06-14 14:36:00,2203-12-23 00:00:00


In [79]:
dec_status_values = ['E', 'S', 'R', 'D', 'L', 'LF', 'AS', 'AR', 'AR IN', 'F', 'AF', 'C', 'ERR']

In [80]:
# Get a df with decayed/de-orbited objects
dec_obj_df = all_obj_df[(all_obj_df['Status'].isin(dec_status_values)) & (all_obj_df['DDate'].notna())]

# Get a df with active objects
act_obj_df = all_obj_df[~all_obj_df['Status'].isin(dec_status_values)]

In [88]:
# Get number of active satellites launched per year
all_obj_df['LaunchYear'] = all_obj_df['LDate'].dt.year
launches_per_yr = all_obj_df.groupby('LaunchYear').size()
all_obj_df[all_obj_df['LaunchYear'] == 1957]

,#JCAT,Satcat,Piece,Type,Name,PLName,LDate,Parent,SDate,Primary,...,Perigee,PF,Apogee,AF,Inc,IF,OpOrbit,OQUAL,AltNames,LaunchYear
1,S00001,1.0,1957 ALP 1,R2,8K71PS No. M1-10 Stage 2,8K71A M1-10 (M1-1PS),1957-10-04,-,1957-10-04 19:33:00,Earth,...,214.0,,938,,65.1,,LLEO/I,-,-,1957.0
2,S00002,2.0,1957 ALP 2,P,1-y ISZ,PS-1,1957-10-04,S00001,1957-10-04 19:33:00,Earth,...,214.0,,938,,65.1,,LLEO/I,-,":RE,:RC",1957.0
3,S00003,3.0,1957 BET 1,P A,2-y ISZ,PS-2,1957-11-03,A00002,1957-11-03 02:35:00,Earth,...,211.0,,1659,,65.33,,LEO/I,-,":RE,:RC",1957.0
1,A00001,NNA,1957 ALP 3,C F,GO,Golovniy Obtekatel',1957-10-04,S00001,1957-10-04 19:33:00,Earth,...,214,,938,,65.10,,LLEO/I,-,-,1957.0
2,A00002,00003,1957 BET 1,R2,8K71PS No. M1-11 Stage 2,8K71A M1-11 (M1-2PS),1957-11-03,-,1957-11-03 02:35:00,Earth,...,211,,1659,,65.33,,LEO/I,-,-,1957.0
3,A00003,NNA,1957 BET 2,C F,GO,Golovniy Obtekatel',1957-11-03,A00002,1957-11-03 02:36:00,Earth,...,211,,1659,,65.33,,LEO/I,-,-,1957.0
8258,A08258,NNA X,1957-U01,PX--V--,USAF 88 Charge A,Poulter Pellet,1957-10-17,R73010,NaT,Earth,...,-2000,?,-20000,?,90.00,,EEO,U,-,1957.0


In [82]:
# Get number of satellites decayed each year
dec_obj_df['DecayYear'] = dec_obj_df['DDate'].dt.year
decays_per_yr = dec_obj_df.groupby('DecayYear').size()

C:\Users\dfarr\AppData\Local\Temp\ipykernel_14656\1194315790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dec_obj_df['DecayYear'] = dec_obj_df['DDate'].dt.year


In [83]:
# Create a dataframe to track sats in orbit each year
years = range(int(all_obj_df['LDate'].dt.year.min()), int(all_obj_df['LDate'].dt.year.max() + 1))
satellites_in_orbit = pd.DataFrame(years, columns=['Year']).set_index('Year')

In [84]:
launches_per_yr.cumsum()

LaunchYear
1958.0        3
1959.0       18
1960.0       35
1961.0      275
1962.0      325
          ...  
2020.0    25527
2021.0    27180
2022.0    30241
2023.0    33225
2024.0    34524
Length: 67, dtype: int64

In [85]:
satellites_in_orbit['Launched'] = launches_per_yr.cumsum()
satellites_in_orbit['Decayed'] = decays_per_yr.cumsum()
satellites_in_orbit = satellites_in_orbit.fillna(0)
satellites_in_orbit['InOrbit'] = satellites_in_orbit['Launched'] - satellites_in_orbit['Decayed']

In [86]:
satellites_in_orbit.sample(5)

,Launched,Decayed,InOrbit
Year,,,
2018,24047.0,23435,612.0
1959,18.0,27,-9.0
1963,479.0,494,-15.0
1986,8134.0,10697,-2563.0
1967,1520.0,1958,-438.0
